In [75]:
from math import ceil

# In the context of 9x1x32 (just one)
def calculate_stalls(I, K, KN, P, S, SA_CONF):
    assert S > 0, f"Stride mustn't be less than or equal to 0"
    BRAM_UNIT_SIZE = 512 #bytes
    DRAM_WRITE_PENALTY = 1 #cycles
    
    IS = (BRAM_UNIT_SIZE - (I[0] + I[1] + 3))
    assert IS > 0, f"Initial Surplus expected to be greater than 0"
    O = (((I[0] - K[0] + 2*P)/S)+1, ((I[1] - K[1] + 2*P)/S)+1)
    
    RQ = I[0] * I[1]
    WQ = O[0] * O[1]
    
    TR = RQ/32
    TW = WQ/32
    
    SM = ceil(IS/DRAM_WRITE_PENALTY)
    TS = ceil(TW/SM) if IS < RQ else 0 
    SP = IS
    TSP = TS * SP * (KN/SA_CONF[1])
    
    print(O)
    print(f"Total Reads Requests: {TR}")
    print(f"Total Write Requests: {TW}")
    print(f"Surplus Moment: every {SM} writes")
    print(f"Initial Surplus: {IS}")
    print(f"Total Stall: {TS}")
    print(f"Total Stall penalty: {(TSP/10**8)*1000}ms")
    return TSP

In [ ]:
I = (224,224)
K = (3,3)
KN = 64
P = 1
S = 1
dims = [(224,224,64), (224,224,64), (112,112,128), (112,112,128), (56,56,256), (56,56,256), (56,56,256), (28,28,512), (28,28,512), (28,28,512), (14,14,512), (14,14,512), (14,14,512)]
tsp_sum = 0
for i in dims:
    tsp_sum = tsp_sum + calculate_stalls((i[0],i[1]), K, i[2], P, S, (9,1))
    print()
print((tsp_sum/10**8)*1000)

In [23]:
# Pipelined conv/sa

def sa_total_size(sa_confs):
    return sum(i[1] * i[2] for i in sa_confs)

def sa_total_brams(dims, sa_confs):
    print("BRAMS")
    sum = 0
    # output brams
    for i in range(len(sa_confs)):
        d = dims[i]
        sc = sa_confs[i]
        brams = ((d[4] * d[5]) * sc[1])/512
        sum = sum + brams
        print(f"{sc}: {brams} output brams")
    # Input brams
    ibrams = 0
    for i in sa_confs:
        ibrams = ibrams + i[2]
    ibrams = ibrams * 9
    print(f"Input brams: {ibrams}")
    sum = sum + ibrams
    print(f"Total brams: {sum} ({1056-sum} left)")
    return sum

def sa_time(dimi, sai):
    return ((dimi[0] * dimi[1]) * (dimi[2]/sai[1]) * ((dimi[3]/sai[2])))/10**5

def check_pipeline(sa_confs):
    for i in range(len(sa_confs)-1):
        if sa_confs[i][1] != sa_confs[i+1][2]:
            return False
    return True
        
def sa_total_time(dims, sa_confs):
    print("TIME")
    if not check_pipeline(sa_confs):
        print("Not pipelineable") 
    sum = 0
    for i in range(len(dims)):
        t = sa_time(dims[i], sa_confs[i % len(sa_confs)])
        sum = sum + t
        print(f"{dims[i]} on {sa_confs[i % len(sa_confs)]}: {t}ms")
    print(f"Time: {sum}")

def summary(dims, sa_confs):
    print(f"Total size SA: 9x{sa_total_size(sa_confs)}")
    sa_total_time(dims,sa_confs)
    sa_total_brams(dims,sa_confs)


In [38]:
#sa_confs = [(9,1,3),(9,8,4),(9,4,6),(9,2,4)]
#sa_confs = [(9,2,3),(9,8,2)]
#sa_confs = [(9,1,8),(9,2,8),(9,2,8),(9,4,4),(9,1,8)]
#sa_confs = [(9,4,16)]
sa_confs = [(9,1,32)]
# dims (ih,iw,kn,ic)
#dims = [(224,224,64,3), (224,224,64,64), (112,112,128,64), (112,112,128,128), (56,56,256,128), (56,56,256,256), (56,56,256,256), (28,28,512,256), (28,28,512,512), (28,28,512,512), (14,14,512,512), (14,14,512,512), (14,14,512,512)]
dims = [(224,224,64,3,224,224), (224,224,64,64,112,112), (112,112,128,64,112,112), (112,112,128,128,56,56), (56,56,256,128,56,56), (56,56,256,256,56,56), (56,56,256,256,28,28), (28,28,512,256,28,28), (28,28,512,512,28,28), (28,28,512,512,14,14), (14,14,512,512,14,14), (14,14,512,512,14,14), (14,14,512,512,14,14)]
dims = [(32,32,64,3,32,32)]
summary(ll, sa_confs)

Total size SA: 9x32
TIME
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
(32, 32, 64, 3, 32, 32) on (9, 1, 32): 0.06144ms
Time: 0.7987200000000002
BRAMS
(9, 1, 32): 2.0 output brams
Input brams: 288
Total brams: 290.0 (766.0 left)


In [36]:
ll = []
for i in range(13):
    ll.append(dims[0])

In [1]:
net_conf = [{"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3},
            {"type":"conv", "pad": 1, "stride": 1, "kernel": 3}]
            
def input_size_propagator(net_conf):
    for i in net_conf:
        print(i)
input_size_propagator(net_conf)
    

{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
{'type': 'conv', 'pad': 1, 'stride': 1, 'kernel': 3}
